### Step 1: Set Up the Environment in Colab

#### Prerequisite: Use L4 GPU
I found the dataset.map step crashes with low memory with T4 TPU

In [14]:
!pip install transformers datasets nltk huggingface_hub torch bitsandbytes evaluate rouge_score

#since my fine tuned model dmrhimali/falcon7binstruct_mentalhealthmodel_oct23 was trained from peft used from source , install peft from source
! pip install git+https://github.com/huggingface/peft.git

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=5f8b6fa5ac0301af6457c5623f559ed661cf9031669af699e8682fe8f6194ba6
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-uete_hid
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-uete_hid
  Resolved https://github.com/huggingface/peft.git to commit ae55fdcc5c4830e0f9fb6e56f16555bafca392de
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Step 2: Import Required Libraries

In [2]:
import torch
from datasets import load_dataset
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
import torch


###  Step 3: Load Your Fine-Tuned Model and Tokenizer

load your fine tuned model and tokenize:

In [3]:
#secret HF_TOKEN is set in colab
# model_name = "dmrhimali/falcon7binstruct_mentalhealthmodel_oct23"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# Loading fine-tuned model from Hugging Face
PEFT_MODEL = "dmrhimali/falcon7binstruct_mentalhealthmodel_oct23" #"Srishy/falcon7binstruct_mentalhealthmodel2_180"
config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)
peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

adapter_config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

### Step 4: Load an Evaluation Dataset


Ensure that your dataset has two fields: the **input** (e.g., the user's message) and **output** (e.g., the chatbot's response).

`Amod/mental_health_counseling_conversations` has `Context`(user's message) and `Response`(Chattbot response)

In [4]:
# take smaller dataset of 10 rows
dataset = load_dataset("Amod/mental_health_counseling_conversations", split="train[:10]") #cold not find a validation split. Should have done `dataset.train_test_split(test_size=0.1)` in splitting original data when training my finetuned model instead of using whole dataset for training

import json
print(dataset)
#sample data item:
print(json.dumps(dataset[0], indent=2))

README.md:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

combined_dataset.json:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Dataset({
    features: ['Context', 'Response'],
    num_rows: 10
})
{
  "Context": "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
  "Response": "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \u00a0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the b

### Step 5: Tokenize and Prepare Data

In [5]:
def preprocess_data(examples):
    # Tokenize inputs and outputs for the conversational model
    input_encodings = peft_tokenizer(examples['Context'], truncation=True, padding=True, return_tensors="pt") #input encoding uses user message field: Context
    return input_encodings

dataset = dataset.map(preprocess_data, batched=True)
print(dataset)
#print(json.dumps(dataset[0], indent=4))

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['Context', 'Response', 'input_ids', 'attention_mask'],
    num_rows: 10
})


### Step 6: Generate Responses with the Fine-Tuned Model

In [10]:

def generate_response(question):
  prompt = f"""
  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.

  ###Question: {question}

  ###Response:

  """
  peft_encoding = peft_tokenizer(prompt, return_tensors="pt").to("cuda:0")
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                      eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                      temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  generated_response = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)
  return {"generated_response": generated_response}

#generated_responses = dataset.map(generate_response, batched=True, batch_size=1, writer_batch_size=1) #for smaller T4 TPU to stop sending entire dataset, but rather chunks of it to stop overwhelming memory
generated_responses = dataset.map(generate_response)
for i in range(10):
    print(f"Input/Question: {dataset[i]['Context']}")
    print(f"Ground Truth: {dataset[i]['Response']}")
    print(f"Generated Response: {generated_responses[i]}\n")

/usr/local/lib/python3.10/dist-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <class 'transformers_modules.vilsonrodrigues.falcon-7b-instruct-sharded.0e7ea20c0bfd0665eaf3835f1efd12a0e8f02d90.modeling_falcon.FalconPreTrainedModel'>.
  StockPickler.save(self, obj, save_persistent_id)
/usr/local/lib/python3.10/dist-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <class 'transformers_modules.vilsonrodrigues.falcon-7b-instruct-sharded.0e7ea20c0bfd0665eaf3835f1efd12a0e8f02d90.modeling_falcon.FalconPreTrainedModel'>: transformers_modules.vilsonrodrigues.falcon-7b-instruct-sharded.0e7ea20c0bfd0665eaf3835f1efd12a0e8f02d90.modeling_falcon.FalconPreTrainedModel has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

Input/Question: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?
Ground Truth: If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow t

### Step 7: Evaluate Generated Responses Using Metrics

Now, let's evaluate the quality of the generated responses. Common metrics for conversational AI evaluation include **BLEU**, **ROUGE**, and **METEOR**.

You can use **ROUGE** (Recall-Oriented Understudy for Gisting Evaluation) to measure the overlap between the generated responses and the ground truth responses. This is a popular metric for text generation tasks.

Following computes ROUGE scores for the generated responses and compares them to the ground truth. ROUGE scores are available for:
- **ROUGE-1** (unigram overlap),
- **ROUGE-2** (bigram overlap),
- **ROUGE-L** (longest common subsequence).

#### 7.1: Evaluate Using ROUGE Score

In [16]:
from evaluate import load
import evaluate


# Load ROUGE metric
rouge = evaluate.load('rouge')

# Define function to compute ROUGE score
def compute_rouge(example):
    predictions = example['generated_response']
    references = example['Response']  # the ground truth response field in evaluation dataset Amod/mental_health_counseling_conversations
    return rouge.compute(predictions=[predictions], references=[references])

# Apply ROUGE computation to the dataset
rouge_scores = generated_responses.map(compute_rouge, batched=False)

# Print out the ROUGE scores for the first few samples
print(json.dumps(rouge_scores[:3], indent=2)) # Display the ROUGE scores for the first three examples


{
  "Context": [
    "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
    "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
    "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of 

#### 7.2: Evaluate Using BLEU Score

BLEU is another popular metric, primarily used for machine translation but also applicable for evaluating text generation tasks like this. You can compute BLEU scores using **NLTK** or the Hugging Face `datasets` library.


In [26]:
from nltk.translate.bleu_score import sentence_bleu
#import evaluate
#bleu = evaluate.load("bleu")

# Define function to compute BLEU score
def compute_bleu(example):
    predictions = example['generated_response']
    references = example['Response'].split()  # Tokenize the ground truth response in evaluation dataset Amod/mental_health_counseling_conversations
    prediction_tokens = predictions.split()
    #score = bleu.compute(predictions=[prediction_tokens], references=[references])
    score = sentence_bleu([references], prediction_tokens)
    return {"bleu_score": score}

# Apply BLEU score computation
bleu_scores = generated_responses.map(compute_bleu, batched=False)

# Display BLEU scores for the first few samples
print(json.dumps(bleu_scores[:3], indent=2))  # Display the BLEU scores for the first three examples

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

{
  "Context": [
    "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
    "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
    "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of 

#### 7.3: Evaluate Using METEOR

The **METEOR** metric accounts for synonymy, stemming, and word order, which can be useful for text generation tasks.

In [23]:
import evaluate

meteor = evaluate.load("meteor")

# Define function to compute METEOR score
def compute_meteor(example):
    prediction = example['generated_response']
    reference = example['Response']
    score = meteor.compute( predictions=[prediction], references=[reference])
    return {"meteor_score": score}

# Apply METEOR score computation
meteor_scores = generated_responses.map(compute_meteor, batched=False)

# Display METEOR scores for the first few samples
print(json.dumps(meteor_scores[:3], indent=2)) # Display the METEOR scores for the first three examples

{
  "Context": [
    "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
    "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
    "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


#### 7.4 Evaluate Using Perplexity

Perplexity is a common metric used to evaluate language models, especially for generative tasks. It measures how well the model predicts the next word in a sequence, where `lower perplexity indicates better performance`.

For a chatbot generation task, `perplexity can give you a sense of how "fluent" the model's responses are`. It’s calculated as the exponentiation of the loss.

In [29]:
import torch

def compute_perplexity(model, tokenizer, dataset):
    model.eval()
    text = dataset['Response']

    encodings = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    inputs = encodings.input_ids
    attention_mask = encodings.attention_mask

    # Forward pass through the model
    with torch.no_grad():
        outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=inputs)
        loss = outputs.loss

    # Perplexity calculation (exponentiate the loss)
    perplexity = torch.exp(loss)
    return perplexity.item()

# Compute perplexity on the test set
perplexity = compute_perplexity(peft_model, peft_tokenizer, dataset)
print(f"Perplexity: {perplexity}")

Perplexity: 1571.0


### Step 8: Summarize and Analyze Results

Once you've computed the evaluation metrics (e.g., ROUGE, BLEU, METEOR), you can summarize and analyze them to see how well your fine-tuned model performs. Look for patterns:
- High ROUGE or BLEU scores suggest your model is generating relevant, coherent responses.
- Low scores may indicate that the model struggles to generate responses that are similar to the ground truth.


In [30]:
print("ROUGE Scores:", rouge_scores)
print("BLEU Scores:", bleu_scores)
print("METEOR Scores:", meteor_scores)

ROUGE Scores: Dataset({
    features: ['Context', 'Response', 'input_ids', 'attention_mask', 'generated_response', 'rouge1', 'rouge2', 'rougeL', 'rougeLsum'],
    num_rows: 10
})
BLEU Scores: Dataset({
    features: ['Context', 'Response', 'input_ids', 'attention_mask', 'generated_response', 'bleu_score'],
    num_rows: 10
})
METEOR Scores: Dataset({
    features: ['Context', 'Response', 'input_ids', 'attention_mask', 'generated_response', 'meteor_score'],
    num_rows: 10
})


### User Evaluation (Human Evaluation)
For conversational AI, **manual evaluation** or **user evaluation** is also highly recommended. This involves asking human evaluators to rate the generated responses on:
- **Fluency** (Is the response grammatically correct?),
- **Relevance** (Is the response relevant to the input?),
- **Engagement** (Does the response encourage further conversation?),
- **Empathy** (If applicable, does the response express empathy or emotional intelligence?).

### Conclusion

You’ve successfully learned how to:
1. Load your fine-tuned conversational AI model and tokenizer.
2. Prepare and tokenize an evaluation dataset.
3. Generate responses from your fine-tuned model.
4. Evaluate the generated responses using metrics like **ROUGE**, **BLEU**, and **METEOR**.
5. Optionally, conduct **user evaluation** for further qualitative analysis.

By evaluating your fine-tuned conversational AI model using both automated metrics and human feedback, you can get a well-rounded view of its performance and identify areas for improvement.